# **One-time Init**

In [ ]:
from os import chdir, environ
environ['TZ'] = 'US/Central'
!nvidia-smi

Wed Dec  8 15:47:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.49       Driver Version: 496.49       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:02:00.0  On |                  N/A |
| 31%   37C    P8    11W / 170W |  10229MiB / 12288MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/google_drive', force_remount=True)
!unlink /content/drive
!ln -s /content/google_drive/MyDrive/ /content/drive

ModuleNotFoundError: ignored

In [ ]:
# For debug
from os import environ
# environ['CUDA_LAUNCH_BLOCKING'] = '1'

# **Starting Work**

In [1]:
from os import chdir, environ
environ['TZ'] = 'US/Central'
chdir('PointCloud/')
# chdir('/content/drive/PointCloud/')

In [2]:
!ls -lt --time-style='+%y-%m-%d %H:%M:%S'
!dir

'ls' is not recognized as an internal or external command,
operable program or batch file.


 Volume in drive C is OS
 Volume Serial Number is 3A4D-1115

 Directory of C:\Users\immor\colab\PointCloud

01/13/2022  02:16    <DIR>          .
01/08/2022  11:58    <DIR>          ..
10/27/2021  09:09            53,575 build_tree
11/01/2021  22:50             3,795 build_tree.cpp
01/07/2022  15:32            20,639 build_tree.py
11/01/2021  22:50             2,769 build_tree_basic.cpp
11/01/2021  22:50             5,065 build_tree_extra.cpp
10/24/2021  09:45             1,969 build_tree_toy.py
01/11/2022  18:11             8,532 dataset.py
12/03/2021  11:10    <DIR>          datasets
09/23/2021  13:13               208 def.sh
09/24/2021  10:16           579,198 dev.ans
10/27/2021  09:09           350,568 dev.out
01/13/2022  02:16            15,128 encoder.py
10/31/2021  11:26             9,195 model.py
10/17/2021  15:20            12,386 model_arrange.py
11/02/2021  20:28            14,641 model_attention.py
10/23/2021  06:33            14,284 model_carry.py
09/30/2021  21:57        

# **Experiment Init**

In [3]:
import json
import h5py
import torch
import os
from encoder import Encoder, MLP
from segment import Segment
from dataset import *
# from encoders.model_cross import Encoder, init_logging, MLP
from build_tree import get_directions, init_directions
import logging

OUTPUT = 'scratch_shapenetpart'

#encoder
model_size = 2 ** 11
sample_layers = 50 # 2
channel = 1
dim = 4096 # 2048
dim_layer0 = 32 # 16
dim_repeat_cut = 5

#decoder 
ancestor_dim = 512

# data
# prefix = "_orig_pca_nosample_x2"
# transform = no_transform
# augment = 2
# no_prealign = True 
# load_pretrain = True
prefix = "_affined_pca_nosample_x10"
transform = affine_transform
augment = 10
no_prealign = False
load_pretrain = False

sample_child_first = False # True in l7s1
rotate_only = False
num_classes = 16
num_parts = 50
DATASET = './datasets/ShapeNetPart'
chaos_limit = 0


In [4]:
global logging_init_flag
logging_init_flag = False

def init_logging(OUTPUT):
    global logging_init_flag
    if logging_init_flag is True:
        return
    logging_init_flag = True

    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        '%(asctime)s - %(levelname)s:\t%(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')

    fh = logging.FileHandler(f"{OUTPUT}/training.log")
    fh.setLevel(logging.INFO)
    fh.setFormatter(formatter)

    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(formatter)

    logger.addHandler(ch)
    logger.addHandler(fh)

init_logging(OUTPUT)
logging.info(f"prefix = {prefix}")
_ = init_directions(chaos_limit, calc_dmap=False)

2022-01-20 01:01:39 - INFO:	prefix = _affined_pca_nosample_x10
2022-01-20 01:01:39 - INFO:	init_directions: # = 3
2022-01-20 01:01:39 - DEBUG:	0: 0.000000 1.000000 0.000000 otho = {1, 2}
2022-01-20 01:01:39 - DEBUG:	1: 1.000000 0.000000 0.000000 otho = {0, 2}
2022-01-20 01:01:39 - DEBUG:	2: 0.000000 0.000000 1.000000 otho = {0, 1}
2022-01-20 01:01:39 - INFO:	basic # = 3
2022-01-20 01:01:39 - INFO:	transforms # = 48


In [5]:
import json
class_parts = json.load(open(f'{DATASET}/id2parts.json'))
class_parts_list = []
part_mapping = []
for i, p in enumerate(class_parts):
    class_parts_list.append(torch.tensor(p).cuda())
    
    mask = torch.zeros(num_parts).cuda().bool()
    mask[p] = True
    class_parts[i] = mask

    pmap = torch.full([num_parts], -1).cuda()
    pmap[p] = torch.arange(len(p)).cuda()
    part_mapping.append(pmap)


class_parts = torch.stack(class_parts, dim=0)


In [6]:
torch.manual_seed(674433238)
def new_model(num_parts=num_parts):
    encoder = Encoder(model_size, sample_layers, dim, OUTPUT, channel=channel, sample_child_first=sample_child_first, dim_layer0=dim_layer0, dim_repeat_cut=dim_repeat_cut)
    model = Segment(ancestor_dim, encoder, num_parts=num_parts).cuda()
    return model

model = new_model()

2022-01-20 01:01:41 - INFO:	self.N = 2048
2022-01-20 01:01:41 - INFO:	self.num_layers = 12
2022-01-20 01:01:41 - INFO:	self.sample_layers = 50
2022-01-20 01:01:41 - INFO:	layer 0 (leaf) # = 2048 odim = 32
2022-01-20 01:01:41 - INFO:	layer 1 (unsampled) # = 1024 odim = 64
2022-01-20 01:01:41 - INFO:	layer 2 (unsampled) # = 512 odim = 128
2022-01-20 01:01:41 - INFO:	layer 3 (unsampled) # = 256 odim = 256
2022-01-20 01:01:41 - INFO:	layer 4 (unsampled) # = 128 odim = 512
2022-01-20 01:01:41 - INFO:	layer 5 (unsampled) # = 64 odim = 512
2022-01-20 01:01:41 - INFO:	layer 6 (unsampled) # = 32 odim = 1024
2022-01-20 01:01:41 - INFO:	layer 7 (unsampled) # = 16 odim = 1024
2022-01-20 01:01:41 - INFO:	layer 8 (unsampled) # = 8 odim = 2048
2022-01-20 01:01:41 - INFO:	layer 9 (unsampled) # = 4 odim = 2048
2022-01-20 01:01:41 - INFO:	layer 10 (unsampled) # = 2 odim = 4096
2022-01-20 01:01:41 - INFO:	layer 11 (unsampled) # = 1 odim = 4096
2022-01-20 01:01:41 - INFO:	layer 0 feature_dim = 4096 globe_

In [7]:
model

Segment(
  (encoder): Encoder(
    (layers): ModuleList(
      (0): EncoderLayer(
        (pts_align): Alignment(
          (conv1): Conv1d(3, 128, kernel_size=(1,), stride=(1,))
          (conv2): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
          (conv3): Conv1d(256, 1024, kernel_size=(1,), stride=(1,))
          (fc1): Linear(in_features=1024, out_features=512, bias=True)
          (fc2): Linear(in_features=512, out_features=256, bias=True)
          (fc3): Linear(in_features=256, out_features=9, bias=True)
          (relu): ReLU()
          (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, 

# **Debug**

In [ ]:
assert False, "debug part cannot be run through"

In [ ]:
trainset = h5py.File(f'{DATASET}/train0.h5')
pts = trainset['data'][0]
pts = torch.tensor(pts)
output = None

<ipython-input-8-068f593c90ea>:1: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  trainset = h5py.File(f'{DATASET}/train0.h5')


In [ ]:
pts = transform(pts).cpu()

In [ ]:
tree = model.encoder.tree
pts, output, extra = tree.arrange(pts, rotate=not no_prealign, extra=False, pca=True,
                                  rotate_only=rotate_only, debug=False)

In [ ]:
batch_size = 64
model.train()
for i in range(8):
    _pts = pts.unsqueeze(0).expand(batch_size, *pts.shape)
    _output = list(map(lambda x : x.unsqueeze(0).expand(batch_size, *x.shape), output))
    _extra = extra.unsqueeze(0).expand(batch_size, *extra.shape)
    feature = model(_pts, _output, _extra)
    print(f"#{i} {feature.shape}")
    (feature.sum()).backward()

#0 torch.Size([64, 2048, 512])
#1 torch.Size([64, 2048, 512])
#2 torch.Size([64, 2048, 512])
#3 torch.Size([64, 2048, 512])
#4 torch.Size([64, 2048, 512])
#5 torch.Size([64, 2048, 512])
#6 torch.Size([64, 2048, 512])
#7 torch.Size([64, 2048, 512])


In [ ]:
%matplotlib inline
def plot(points, color_layers=2, output=None, model=model.encoder):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.gridspec as gridspec
    import numpy as np

    points = points.cpu().numpy()
    x, y, z = points[:,0], points[:,1], points[:,2]
    x -= x.mean()
    y -= y.mean()
    z -= z.mean()

    color = np.array([0 for _ in range(x.shape[0])])
    
    def mark_color(z, last_ind=None):
        n = z.shape[0]
        ind = torch.tensor(z).sort()[1].numpy()
        indl, indr = ind[: n >> 1], ind[n >> 1 : ]
        if last_ind is not None:
            indl, indr = last_ind[indl], last_ind[indr]
        color[indr] += 1
        return indl, indr

    if output is not None:

        color = torch.tensor([0]).cuda()
        # print(tree.layer_size)
        for i, (layer, next_layer_size) in enumerate(zip(reversed(model.layers[1:]), tree.layer_size[1:])):
            next_color = torch.zeros(next_layer_size).long().cuda()
            if i < color_layers:
                color = color << 1
                right_add = 1
            else:
                right_add = 0
            assert layer.child_l.max() < next_layer_size
            assert layer.child_r.max() < next_layer_size
            next_color[layer.child_l] = color
            next_color[layer.child_r] = color + right_add
            color = next_color

        arrange = output[0]
        n = points.shape[0]
        color = np.array([color[arrange == i].max().item() for i in range(n)])

    elif color_layers >= 1:
        l, r = mark_color(z)
        if color_layers >= 2:
            color *= 2
            ll, lr = mark_color(y[l], l)
            rl, rr = mark_color(y[r], r)
            if color_layers >= 3:
                color *= 2
                for p in [ll, lr, rl, rr]:
                    mark_color(x[p], p)

    colormap = np.array(['red', 'blue', 'green', 'yellow', 'grey', 'orange', 'purple', 'cyan'])

    fig = plt.figure(dpi=80)
    gs = gridspec.GridSpec(nrows=2, ncols=4, left=0.1, right=2.5, wspace=0.05, hspace=0.05, bottom=0.1, top=1.3)
    for i in range(8):
        ax = fig.add_subplot(gs[i // 4, i % 4], projection='3d')

        x, y, z = points[:,0], points[:,1], points[:,2]
        labx, laby, labz = 'x', 'y', 'z'
        if (i & 4) != 0:    x = -x; labx = '-x'
        if (i & 2) != 0:    y = -y; laby = '-y'
        if (i & 1) != 0:    z = -z; labz = '-z'

        lmin = min(x.min(), y.min(), z.min())
        lmax = max(x.max(), y.max(), z.max())
        ax.scatter(x, y, z, c=colormap[color], marker='.')

        ax.set_xlim(lmin, lmax)
        ax.set_ylim(lmin, lmax)
        ax.set_zlim(lmin, lmax)

        ax.set_xlabel(labx)
        ax.set_ylabel(laby)
        ax.set_zlabel(labz) 
    plt.show()

    fig = plt.figure(dpi=80)
    ncolor = color.max() + 1
    nrows = max(1, ncolor // 4)
    ncols = min(ncolor, 4)
    gs = gridspec.GridSpec(nrows=nrows, ncols=ncols, left=0.1, right=0.1 + 0.6 * ncols, wspace=0.05, hspace=0.05, bottom=0.1, top=0.1 + 0.6 * nrows)
    for i in range(ncolor):
        ax = fig.add_subplot(gs[i // 4, i % 4], projection='3d')

        p = points[color == i]
        x, y, z = p[:,0], p[:,1], p[:,2]

        lmin = min(x.min(), y.min(), z.min())
        lmax = max(x.max(), y.max(), z.max())
        ax.scatter(x, y, z, c=colormap[i], marker='.')

        ax.set_xlim(lmin, lmax)
        ax.set_ylim(lmin, lmax)
        ax.set_zlim(lmin, lmax)

        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z') 
    plt.show()

In [ ]:
plot(pts, 3, output=output)

In [ ]:
pts = train_data[7][0]
output = None

In [ ]:
for i in range(50):
    output, arrange = tree.arrange(pts, basic=basic, debug=False, device='cpu')

In [ ]:
feature.shape

In [ ]:
for i in range(10):
    loss = model(pts.unsqueeze(0), list(map(lambda x : x.unsqueeze(0), output))).view(-1).sum()
    loss.backward()

In [ ]:
model

In [ ]:
for name, param in model.named_parameters():
    print(name, param.size())

In [ ]:
for name, param in model.named_parameters():
    if 'activate' in name:
        print(name, "\t", param.item())

In [ ]:
!cp /tmp/cppinput.txt scratch/

In [ ]:
!tail -n 5 /tmp/cppoutput.txt

# **Data**

In [8]:
from dataset import *
import numpy as np
from math import ceil

In [9]:
make = make_data_default
if rotate_only:
    make = make_data_rotate_only
if no_prealign:
    make = make_data_no_prealign

In [10]:
n_train = 6
clouds = []
labels = []
extra_labels = []
for i in range(n_train):
    data_file = h5py.File(f'{DATASET}/train{i}.h5')
    clouds.append(torch.tensor(np.array(data_file['data'])))
    labels.append(torch.tensor(np.array(data_file['seg'])))
    extra_labels.append(torch.tensor(np.array(data_file['label'])))

clouds = torch.cat(clouds, dim=0)
labels = torch.cat(labels, dim=0)
extra_labels = torch.cat(extra_labels, dim=0)

train_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=augment, transform=transform, make=make, extra_labels=extra_labels)


<ipython-input-10-95d4cf321aff>:6: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data_file = h5py.File(f'{DATASET}/train{i}.h5')


In [11]:
n_valid = 1
clouds = []
labels = []
extra_labels = []
for i in range(n_valid):
    data_file = h5py.File(f'{DATASET}/val{i}.h5')
    clouds.append(torch.tensor(np.array(data_file['data'])))
    labels.append(torch.tensor(np.array(data_file['seg'])))
    extra_labels.append(torch.tensor(np.array(data_file['label'])))

clouds = torch.cat(clouds, dim=0)
labels = torch.cat(labels, dim=0)
extra_labels = torch.cat(extra_labels, dim=0)

valid_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=max(1, int(ceil(augment/4))), transform=transform, make=make, extra_labels=extra_labels)

<ipython-input-11-e65829196ec3>:6: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data_file = h5py.File(f'{DATASET}/val{i}.h5')


In [12]:
n_test = 2
clouds = []
labels = []
extra_labels = []
for i in range(n_test):
    data_file = h5py.File(f'{DATASET}/test{i}.h5')
    clouds.append(torch.tensor(np.array(data_file['data'])))
    labels.append(torch.tensor(np.array(data_file['seg'])))
    extra_labels.append(torch.tensor(np.array(data_file['label'])))

clouds = torch.cat(clouds, dim=0)
labels = torch.cat(labels, dim=0)
extra_labels = torch.cat(extra_labels, dim=0)

test_dataset = PointCloudDataset(clouds, labels, model.tree.arrange, augment=1, make=make, transform=transform, extra_labels=extra_labels)

<ipython-input-12-00e98f6a9ccf>:6: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  data_file = h5py.File(f'{DATASET}/test{i}.h5')


In [13]:
len(train_dataset), len(valid_dataset), len(test_dataset)

(121370, 5610, 2874)

In [14]:
target = [] # ['train', 'valid', 'test']

import gc
gc.collect()

for name, dataset in zip(['train', 'valid', 'test'], [train_dataset, valid_dataset, test_dataset]):
    from tqdm import tqdm

    if name not in target:
        continue

    num_workers = 16
    batch_size = 16

    data_init = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=placeholder, pin_memory=False, drop_last=False)
    logging.info(f"Init {name}")

    mem = []

    def save():
        torch.save(mem, f'{OUTPUT}/{name}_data{prefix}.pth')
        # logging.info("Saved")

    for i, data in enumerate(tqdm(data_init)):
        mem += data
        # logging.debug(f"Init {name}: {i+1}/{len(data_init)}")

        if (i + 1) % (400 * max(1, augment//4)) == 0:
            save()
    
    save()
    logging.info("Done")

    del mem
    gc.collect()


# **Pretrain**

In [ ]:
assert False, "not implemented"

In [ ]:
class FeatureMap(torch.nn.Module):
    def __init__(self, dim):
        super(FeatureMap, self).__init__()
        self.relu = torch.nn.ReLU()

        self.linear1 = torch.nn.Linear(dim, dim // 2)
        self.bn1 = torch.nn.BatchNorm1d(dim // 2)

        self.linear2 = torch.nn.Linear(dim // 2, dim // 4)
        self.bn2 = torch.nn.BatchNorm1d(dim // 4)

        self.linear3 = torch.nn.Linear(dim // 4, dim // 8)
        self.bn3 = torch.nn.BatchNorm1d(dim // 8)
        self.dropout = torch.nn.Dropout(0.7)

    def forward(self, ans):
        ans = self.linear1(ans)
        ans = self.bn1(ans)
        ans = self.relu(ans)

        ans = self.linear2(ans)
        ans = self.dropout(ans)
        ans = self.bn2(ans)
        ans = self.relu(ans)
        
        ans = self.linear3(ans)
        ans = self.bn3(ans)
        return ans

torch.manual_seed(674433238)
feature_map = FeatureMap(dim).cuda()

In [ ]:
try:
    raw_data
except:
    raw_data = torch.load(f'{OUTPUT}/train_data{prefix}.pth')
try:
    test_data
except:
    test_data = torch.load(f'{OUTPUT}/test_data{prefix}.pth')
try:
    train_perm, valid_perm = torch.load(f'{OUTPUT}/train_split{prefix}.pth')
except:
    train_perm, valid_perm = torch.load(f'{OUTPUT}/train_split.pth')

n = len(raw_data)

train_data = [raw_data[i * data_augments + j] for i in train_perm for j in range(data_augments)]
valid_data = [raw_data[i * data_augments + j] for i in valid_perm for j in range(data_augments)]

classes = [[] for i in range(num_classes)]
for (pts, output, extra, label) in train_data:
    classes[label.item()].append((pts, output, extra, label))

valid_classes = [[] for i in range(num_classes)]
for (pts, output, extra, label) in valid_data:
    valid_classes[label.item()].append((pts, output, extra, label))

from random import shuffle, seed
seed(674433238)
shuffle(train_data)
shuffle(valid_data)
shuffle(test_data)

In [ ]:
def make_batch(batches, eval=False):
    from random import randint
    layers = 0
    points = []
    inputs = None
    labels = []
    extras = []
    for pts, output, extra, label in batches:
        # Simple data augment
        # sgn = torch.tensor(-1).pow(torch.randint(low=0, high=2, size=[extra.shape[-1]])).cuda()
        sgn = (-1) ** randint(0, 1)
        
        points.append(pts.cuda())
        labels.append(label.cuda())
        extras.append(extra.cuda() * (1 if eval else sgn))
        if inputs is None:
            layers = len(output)
            inputs = [[] for _ in output]

        for line, out in zip(inputs, output):
            line.append(out.cuda())

    points = torch.stack(points, dim=0).float().cuda()
    extras = torch.stack(extras, dim=0).float().cuda()
    for i, line in enumerate(inputs):
        inputs[i] = torch.stack(line, dim=0).long().cuda()
    labels = torch.stack(labels, dim=0).long().cuda()
    
    return (points, inputs, extras), labels

In [ ]:
from random import choice
num_samples = 2
temperature = 0.07

class_size = torch.tensor([len(c) for c in classes]).float().cuda()
contrast_crit = torch.nn.CrossEntropyLoss(reduce=None)

def contrast_batch(used_classes=classes):
    batches = []
    for _ in range(num_samples):
        for c in used_classes:
            batches.append(choice(c))
    return make_batch(batches)

def contrast_loss(features, labels):
    num_data = num_classes * num_samples

    ratio = class_size[labels]
    ratio /= ratio.sum()
    labels = (labels.unsqueeze(0) == labels.unsqueeze(1))

    features = torch.nn.functional.normalize(features, dim=1)

    sim = features.matmul(features.T)

    mask = torch.eye(num_data).bool().cuda()
    labels = labels[~mask].view(num_data, -1)
    sim = sim[~mask].view(num_data, -1)

    pos = sim[labels].view(num_data, -1)
    neg = sim[~labels].view(num_data, -1)

    logits = torch.cat([pos, neg], dim=1) / temperature

    correct = ((logits.argmax(dim=-1) == 0) * ratio).sum().item() * num_data

    loss = (contrast_crit(logits, torch.zeros(num_data).long().cuda()) * ratio).sum()

    return loss, correct

def valid_eval(classes=valid_classes):
    model.eval()
    feature_map.eval()

    with torch.no_grad():

        cum_loss = 0
        cum_correct = 0
        cum_1 = 0

        for _ in range(8):
            input, labels = contrast_batch(classes)
            features = feature_map(model(*input))
            l, c = contrast_loss(features, labels)

            cum_loss += l.item()
            cum_correct += c / (num_classes * num_samples)
            cum_1 += 1

        valid_str = "valid = %.4lf (%.6lf)" % (cum_loss / cum_1, cum_correct / cum_1)

    model.train()
    feature_map.train()

    return valid_str


In [ ]:
from random import choice, randint, seed
import build_tree
import torch
from math import sin, acos

logging.info(f"train_data # = {len(train_data)}")


model.train()
feature_map.train()
activate = lambda x : x

pi = acos(-1)
cum_loss = 0

batch_scale = 1 # 1 if basic else 1
epoch_scale = 4 # 4 if basic else 2
num_trans = 1

print_epoch = 50
valid_epoch = 1
epoch_since = 0
save_epoch = 500
cut_epoch = 10000000000
correct = 0
total = 0

debug_print = 5

def erate_value(epoch):
    N = 20000
    return torch.tensor(sin(min(1, epoch / N) * pi / 2)).cuda()

threshold = -1.0

num_epoch = 100000

crit = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(list(model.parameters()) + list(feature_map.parameters()), lr=1e-4)
# sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=num_epoch / 5, eta_min=1e-5)
sch = torch.optim.lr_scheduler.ExponentialLR(opt, 0.9999)

def get_trans(n=3):
    if n == 0:  return [None]
    return [randint(0, len(build_tree.transforms) - 1) for _ in range(n)]

In [ ]:
seed(674433238)
torch.manual_seed(674433238)

best_vres = -1.0

for epoch in range(1, num_epoch + 1):
    for _ in range(epoch_scale):
        loss = 0.
        for _ in range(batch_scale):
            input, labels = contrast_batch()
            for iperm in get_trans(num_trans):

                features = feature_map(model(*input, perm=iperm))
                l, c = contrast_loss(features, labels)
                loss += l

                with torch.no_grad():
                    correct += c
                    total += labels.shape[0]
                    epoch_since += 1
        
        # assert loss.isnan().sum() == 0
        cum_loss += loss.item()
        opt.zero_grad()
        loss.backward()
        opt.step()
        
    sch.step()
        
    if epoch <= debug_print or epoch % print_epoch == 0:
        valid_str = ""
        func = logging.debug
        func(f"train #{epoch} lr = {'%.2e' % sch.get_last_lr()[0]} train = {'%.4lf (%.6lf)' % (cum_loss / epoch_since, correct / total)} {valid_eval()}")
        epoch_since = cum_loss = correct = total = 0

    if epoch % save_epoch == 0 or epoch == debug_print:
        torch.save({
            'encoder': model.state_dict(),
            'feature_map': feature_map.state_dict(),
        }, f"{OUTPUT}/pretrained_{epoch}.pth")
        logging.info(f"Saved")

# **Train**

In [ ]:
from dataset import make_batch_train, make_batch_eval
import torch.nn as nn

global batch_size
num_workers = 0
batch_size = 64

for name, dataset in zip(['train', 'valid', 'test'], [train_dataset, valid_dataset, test_dataset]):
    try:
        dataset.mem[0]
    except:
        logging.info(f"Loading {name} dataset's mem")
        dataset.mem = torch.load(f'{OUTPUT}/{name}_data{prefix}.pth')


2021-12-20 01:40:02 - INFO:	Loading train dataset's mem
2021-12-20 01:40:34 - INFO:	Loading valid dataset's mem
2021-12-20 01:40:36 - INFO:	Loading test dataset's mem


In [ ]:
class AccStat:
    def __init__(self):
        self.clear()

    def clear(self):
        self.correct = 0
        self.total = 0

    def add(self, cor, num=1, mean=False):
        if mean:
            cor *= num
        self.correct += cor
        self.total += num

    def result(self, clear=False):
        ret = self.correct / max(self.total, 1e-5)
        if clear:
            self.clear()
        return ret 

    def __str__(self):
        return "%.4lf" % self.result()

def miou(logits, part_label):
    if len(logits.shape) < 3:
        logits = logits.unsqueeze(0)
    
    if len(part_label.shape) < 2:
        part_label = part_label.unsqueeze(0)

    pred = logits.cuda().argmax(dim=-1)
    part = torch.arange(logits.size(-1)).cuda()

    in_pred = part[None, :, None] == pred[:, None, :]
    in_label = part[None, :, None] == part_label.cuda()[:, None, :]

    I = (in_pred & in_label).sum(dim=-1)
    U = (in_pred | in_label).sum(dim=-1)

    part_IOU = I / U.clamp(min=1).float()
    part_IOU[U < 0.5] = 1
    # counted = in_label.sum(dim=-1) > 0.5
    # counted = class_parts[cloud_label.long().cuda()]
    
    # appeared = in_label.sum(dim=-1) > 0.5
    # if (counted != appeared).sum().item() != 0:
    #     for cl, cnt, app in zip(cloud_label, counted, appeared):
    #         if (cnt != app).sum().item() != 0:
    #             arr = torch.arange(num_parts).cuda()
    #             print(f"Error cnt = {arr[cnt]} app = {arr[app]}")
    #     assert False

    shape_IOU = part_IOU.mean(dim=-1)
    return shape_IOU.mean().item()

def evaluate(model, linear, loader, noprint=False, perms=[None]):
    cls = AccStat()
    stat = AccStat()
    trad = AccStat()

    if not noprint:
        logging.info(f"loader # = {len(loader)}")

    print_epoch = 1

    model.eval()
    linear.eval()
    activate = lambda x : x

    for epoch, (input, part_label, cloud_label) in enumerate(loader):
        part_label = part_label.cuda()
        cloud_label = cloud_label.squeeze(-1).cuda()
        with torch.no_grad():
            for iperm in perms:
                features = linear(model(*input, perm=iperm))
                class_index = cloud_label.unique().item()

                logits = model.part_classfier(features)
                part_label = part_mapping[class_index][part_label]

                stat.add(miou(logits, part_label))

        if not noprint:
            if (epoch // batch_size + 1) % print_epoch == 0:
                logging.debug(f"test #{epoch} correct = {'%.6lf' % stat.result()}")

    if not noprint:
        logging.info(f"Done: score = {'%.8lf' % stat.result()}")

    model.train()
    linear.train()  
    
    return stat.result(), f"{stat}"


In [ ]:
import torch.nn.functional as F
global current_epoch
global best_vres
current_epoch = 0
best_vres = -1.0

def train(class_index, more_epoch=100000, valid_result_threshold=999.0):
    from random import choice, randint
    import build_tree
    import torch
    
    model = new_model(len(class_parts_list[class_index]))
    linear = nn.Identity()
    model.train()
    linear.train()
    activate = lambda x : x

    if load_pretrain:
        ckpt = torch.load(f"{OUTPUT}/trained_{class_index}_best_affined_pca_nosample_x10.pth")
        model.load_state_dict(ckpt['model'])
        logging.info("Pretrain loaded")


    def get_trans(n=3):
        if n == 0:  return [None]
        return [randint(0, len(build_tree.transforms) - 1) for _ in range(n)]

    def save(epoch):
        torch.save({
            'model': model.state_dict(),
            # 'linear': linear.state_dict(),
            # 'opt': opt.state_dict(),
            # 'sch': sch.state_dict(),
            # 'best_vres': best_vres,
        }, f"{OUTPUT}/trained_{class_index}_{epoch}.pth")

    global current_epoch
    global best_vres
    global batch_size 

    subset_crit = lambda x : x[-1].item() == class_index
    train_loader = torch.utils.data.DataLoader(Subset(train_dataset, subset_crit), batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=make_batch_train, pin_memory=True, drop_last=True)
    valid_loader = torch.utils.data.DataLoader(Subset(valid_dataset, subset_crit), batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=make_batch_eval, pin_memory=True, drop_last=False)
    test_loader = torch.utils.data.DataLoader(Subset(test_dataset, subset_crit), batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=make_batch_eval, pin_memory=True, drop_last=False)

    logging.info(f"train class_index = {class_index} epoch = {current_epoch + 1} ~ {more_epoch} threshold = {valid_result_threshold} count = {len(train_loader) * batch_size} num_parts = {len(class_parts_list[class_index])}")

    cum_loss = 0

    batch_scale = 1 # 1 if basic else 1
    epoch_scale = max(1, 256 // batch_size) # 8 # 4 if basic else 2
    num_trans = 0 if no_prealign else 1


    print_epoch = 20
    valid_epoch = 20
    epoch_since = 0
    save_epoch = 500
    cut_epoch = 10000000000

    cloud_accuracy = AccStat()
    accuracy = AccStat()
    trad_accuracy = AccStat()
    cloud_classify_coef = 0.3
    part_classify_coef = 0.2
    class_spec_coef = 1

    threshold = -1.0

    crit = torch.nn.CrossEntropyLoss(ignore_index=-1)
    opt = torch.optim.Adam(list(model.parameters()) + list(linear.parameters()), lr=1e-4)
    # sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=num_epoch / 5, eta_min=1e-5)
    sch = torch.optim.lr_scheduler.ExponentialLR(opt, 0.9999)
    save(current_epoch)

    for epoch in range(current_epoch + 1, current_epoch + more_epoch + 1):

        current_epoch = epoch

        for _ in range(epoch_scale):
            loss = 0.
            for _ in range(batch_scale):
                input, part_label, cloud_label = next(iter(train_loader))
                part_label = part_label.cuda()
                cloud_label = cloud_label.squeeze(-1).cuda()
                assert cloud_label.unique().item() == class_index
                for iperm in get_trans(num_trans):
                    
                    features = linear(model(*input, perm=iperm))

                    # def get_parts_weight(plabel, num_parts=num_parts, parts=None):
                    #     plabel = plabel.cuda()
                    #     if parts is None:
                    #         parts = torch.arange(num_parts, device='cuda')
                        
                    #     parts_cnt = (parts[:, None] == plabel.reshape(-1)[None, :]).sum(dim=-1)
                    #     parts_weight = 1 / parts_cnt.clamp(min=1)
                    #     parts_weight[parts_cnt < 0.5] = 0
                    #     parts_weight /= parts_weight.sum()
                    #     return parts_weight

                    logits = model.part_classfier(features)
                    part_label = part_mapping[class_index][part_label]
                    # weight = get_parts_weight(part_label, num_parts=len(class_parts_list[class_index]))
                    loss += F.cross_entropy(logits.transpose(-1, -2), part_label) * class_spec_coef

                    with torch.no_grad():
                        accuracy.add(miou(logits, part_label))
                        epoch_since += 1
            
            assert loss.isnan().sum() == 0
            cum_loss += loss.item()
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        sch.step()

        if cum_loss / epoch_since < threshold:
            epoch_scale, batch_scale = batch_scale, epoch_scale
            
            logging.info("Threshold Reached")
            threshold = -1e10
            
        if epoch <= 5 or epoch % print_epoch == 0:
            valid_str = ""
            func = logging.debug

            stop_training = False
            if epoch % valid_epoch == 0:
                vres, valid_str = evaluate(model, linear, valid_loader, noprint=True)
                valid_str = "valid = " + valid_str
                stop_training = (vres >= valid_result_threshold)
                if vres > best_vres:
                    best_vres = vres
                    save(f"best{prefix}")
                    valid_str += " updated"

                func = logging.info
            func(f"train #{epoch} lr = {'%.2e' % sch.get_last_lr()[0]} loss = {'%.6lf' % (cum_loss / epoch_since)} train = {accuracy} {valid_str}")
            epoch_since = cum_loss = 0
            accuracy.clear()
            cloud_accuracy.clear()
            trad_accuracy.clear()

            if stop_training:
                break


        if epoch % save_epoch == 0:
            save(epoch)
            tres, test_str = evaluate(model, linear, test_loader, noprint=True)
            logging.info(f"Saved test = {test_str}")

        if epoch % cut_epoch == 0:
            if batch_size > 8:
                batch_size //= 2
                epoch_scale *= 2

            logging.info(f"Cut batch_size = {batch_size} epoch_scale = {epoch_scale}")


# **Train Result Archive**

In [ ]:
assert False, "protected"

AssertionError: ignored

In [ ]:
# affined
train(0)

2021-12-08 14:21:39 - INFO:	train class_index = 0 epoch = 1 ~ 100000 threshold = 1.0
2021-12-08 14:21:40 - INFO:	self.N = 2048
2021-12-08 14:21:40 - INFO:	self.num_layers = 12
2021-12-08 14:21:40 - INFO:	self.sample_layers = 50
2021-12-08 14:21:40 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-08 14:21:40 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-08 14:21:40 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-08 14:21:40 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-08 14:21:40 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-08 14:21:40 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-08 14:21:40 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-08 14:21:40 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-08 14:21:40 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-08 14:21:40 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-08 14:21:40 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-08 14:21:40 - INFO:	layer 11 (unsample

KeyboardInterrupt: ignored

In [ ]:
# affined
train(1)

2021-12-19 15:58:08 - INFO:	self.N = 2048
2021-12-19 15:58:08 - INFO:	self.num_layers = 12
2021-12-19 15:58:08 - INFO:	self.sample_layers = 50
2021-12-19 15:58:08 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-19 15:58:08 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-19 15:58:08 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-19 15:58:08 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-19 15:58:08 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-19 15:58:08 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-19 15:58:08 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-19 15:58:08 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-19 15:58:08 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-19 15:58:08 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-19 15:58:08 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-19 15:58:08 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-19 15:58:08 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(2)

2021-12-19 18:40:49 - INFO:	self.N = 2048
2021-12-19 18:40:49 - INFO:	self.num_layers = 12
2021-12-19 18:40:49 - INFO:	self.sample_layers = 50
2021-12-19 18:40:49 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-19 18:40:49 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-19 18:40:49 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-19 18:40:49 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-19 18:40:49 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-19 18:40:49 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-19 18:40:49 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-19 18:40:49 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-19 18:40:49 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-19 18:40:49 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-19 18:40:49 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-19 18:40:49 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-19 18:40:49 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affine
train(3)

2021-12-17 09:38:50 - INFO:	self.N = 2048
2021-12-17 09:38:50 - INFO:	self.num_layers = 12
2021-12-17 09:38:50 - INFO:	self.sample_layers = 50
2021-12-17 09:38:50 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-17 09:38:50 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-17 09:38:50 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-17 09:38:50 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-17 09:38:50 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-17 09:38:50 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-17 09:38:50 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-17 09:38:50 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-17 09:38:50 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-17 09:38:50 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-17 09:38:50 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-17 09:38:50 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-17 09:38:50 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(4)

2021-12-09 00:47:47 - INFO:	train class_index = 4 epoch = 1 ~ 100000 threshold = 999.0 count = 26560 num_parts = 4
2021-12-09 00:47:47 - INFO:	self.N = 2048
2021-12-09 00:47:47 - INFO:	self.num_layers = 12
2021-12-09 00:47:47 - INFO:	self.sample_layers = 50
2021-12-09 00:47:47 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-09 00:47:47 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-09 00:47:47 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-09 00:47:47 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-09 00:47:47 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-09 00:47:47 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-09 00:47:47 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-09 00:47:47 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-09 00:47:47 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-09 00:47:47 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-09 00:47:47 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-09 00:47

KeyboardInterrupt: ignored

In [ ]:
# affined
train(5)

2021-12-19 20:19:11 - INFO:	self.N = 2048
2021-12-19 20:19:11 - INFO:	self.num_layers = 12
2021-12-19 20:19:11 - INFO:	self.sample_layers = 50
2021-12-19 20:19:11 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-19 20:19:11 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-19 20:19:11 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-19 20:19:11 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-19 20:19:11 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-19 20:19:11 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-19 20:19:11 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-19 20:19:11 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-19 20:19:11 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-19 20:19:11 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-19 20:19:11 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-19 20:19:11 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-19 20:19:11 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(6)

2021-12-17 15:50:05 - INFO:	self.N = 2048
2021-12-17 15:50:05 - INFO:	self.num_layers = 12
2021-12-17 15:50:05 - INFO:	self.sample_layers = 50
2021-12-17 15:50:05 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-17 15:50:05 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-17 15:50:05 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-17 15:50:05 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-17 15:50:05 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-17 15:50:05 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-17 15:50:05 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-17 15:50:05 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-17 15:50:05 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-17 15:50:05 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-17 15:50:05 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-17 15:50:05 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-17 15:50:05 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(7)

2021-12-19 00:53:02 - INFO:	self.N = 2048
2021-12-19 00:53:02 - INFO:	self.num_layers = 12
2021-12-19 00:53:02 - INFO:	self.sample_layers = 50
2021-12-19 00:53:02 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-19 00:53:02 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-19 00:53:02 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-19 00:53:02 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-19 00:53:02 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-19 00:53:02 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-19 00:53:02 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-19 00:53:02 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-19 00:53:02 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-19 00:53:02 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-19 00:53:02 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-19 00:53:02 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-19 00:53:02 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(8)

2021-12-16 22:01:24 - INFO:	self.N = 2048
2021-12-16 22:01:24 - INFO:	self.num_layers = 12
2021-12-16 22:01:24 - INFO:	self.sample_layers = 50
2021-12-16 22:01:24 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-16 22:01:24 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-16 22:01:24 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-16 22:01:24 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-16 22:01:24 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-16 22:01:24 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-16 22:01:24 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-16 22:01:24 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-16 22:01:24 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-16 22:01:24 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-16 22:01:24 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-16 22:01:24 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-16 22:01:24 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(9)

2021-12-18 15:06:09 - INFO:	self.N = 2048
2021-12-18 15:06:09 - INFO:	self.num_layers = 12
2021-12-18 15:06:09 - INFO:	self.sample_layers = 50
2021-12-18 15:06:10 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-18 15:06:10 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-18 15:06:10 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-18 15:06:10 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-18 15:06:10 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-18 15:06:10 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-18 15:06:10 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-18 15:06:10 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-18 15:06:10 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-18 15:06:10 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-18 15:06:10 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-18 15:06:10 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-18 15:06:10 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(10)

2021-12-17 23:37:00 - INFO:	self.N = 2048
2021-12-17 23:37:00 - INFO:	self.num_layers = 12
2021-12-17 23:37:00 - INFO:	self.sample_layers = 50
2021-12-17 23:37:00 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-17 23:37:00 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-17 23:37:00 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-17 23:37:00 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-17 23:37:00 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-17 23:37:00 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-17 23:37:00 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-17 23:37:00 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-17 23:37:00 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-17 23:37:00 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-17 23:37:00 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-17 23:37:00 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-17 23:37:00 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(11)

2021-12-19 22:27:51 - INFO:	self.N = 2048
2021-12-19 22:27:51 - INFO:	self.num_layers = 12
2021-12-19 22:27:51 - INFO:	self.sample_layers = 50
2021-12-19 22:27:51 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-19 22:27:51 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-19 22:27:51 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-19 22:27:51 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-19 22:27:51 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-19 22:27:51 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-19 22:27:51 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-19 22:27:51 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-19 22:27:51 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-19 22:27:51 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-19 22:27:51 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-19 22:27:51 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-19 22:27:51 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(12)

2021-12-19 10:04:45 - INFO:	self.N = 2048
2021-12-19 10:04:45 - INFO:	self.num_layers = 12
2021-12-19 10:04:45 - INFO:	self.sample_layers = 50
2021-12-19 10:04:45 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-19 10:04:45 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-19 10:04:45 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-19 10:04:45 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-19 10:04:45 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-19 10:04:45 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-19 10:04:45 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-19 10:04:45 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-19 10:04:45 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-19 10:04:45 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-19 10:04:45 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-19 10:04:45 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-19 10:04:45 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(13)

2021-12-17 21:34:07 - INFO:	self.N = 2048
2021-12-17 21:34:07 - INFO:	self.num_layers = 12
2021-12-17 21:34:07 - INFO:	self.sample_layers = 50
2021-12-17 21:34:07 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-17 21:34:07 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-17 21:34:07 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-17 21:34:07 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-17 21:34:07 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-17 21:34:07 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-17 21:34:07 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-17 21:34:07 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-17 21:34:07 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-17 21:34:07 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-17 21:34:07 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-17 21:34:07 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-17 21:34:07 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(14)

2021-12-20 01:40:36 - INFO:	self.N = 2048
2021-12-20 01:40:36 - INFO:	self.num_layers = 12
2021-12-20 01:40:36 - INFO:	self.sample_layers = 50
2021-12-20 01:40:36 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-20 01:40:36 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-20 01:40:36 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-20 01:40:36 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-20 01:40:36 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-20 01:40:36 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-20 01:40:36 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-20 01:40:36 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-20 01:40:36 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-20 01:40:36 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-20 01:40:36 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-20 01:40:36 - INFO:	layer 11 (unsampled) # = 1 odim = 2048
2021-12-20 01:40:36 - INFO:	layer 0 feature_dim = 2048 globe_dim

KeyboardInterrupt: ignored

In [ ]:
# affined
train(15)

2021-12-09 10:09:34 - INFO:	train class_index = 15 epoch = 1 ~ 100000 threshold = 999.0 count = 38336 num_parts = 3
2021-12-09 10:09:34 - INFO:	self.N = 2048
2021-12-09 10:09:34 - INFO:	self.num_layers = 12
2021-12-09 10:09:34 - INFO:	self.sample_layers = 50
2021-12-09 10:09:34 - INFO:	layer 0 (leaf) # = 2048 odim = 16
2021-12-09 10:09:34 - INFO:	layer 1 (unsampled) # = 1024 odim = 32
2021-12-09 10:09:34 - INFO:	layer 2 (unsampled) # = 512 odim = 64
2021-12-09 10:09:34 - INFO:	layer 3 (unsampled) # = 256 odim = 128
2021-12-09 10:09:34 - INFO:	layer 4 (unsampled) # = 128 odim = 256
2021-12-09 10:09:34 - INFO:	layer 5 (unsampled) # = 64 odim = 256
2021-12-09 10:09:34 - INFO:	layer 6 (unsampled) # = 32 odim = 512
2021-12-09 10:09:34 - INFO:	layer 7 (unsampled) # = 16 odim = 512
2021-12-09 10:09:34 - INFO:	layer 8 (unsampled) # = 8 odim = 1024
2021-12-09 10:09:34 - INFO:	layer 9 (unsampled) # = 4 odim = 1024
2021-12-09 10:09:34 - INFO:	layer 10 (unsampled) # = 2 odim = 2048
2021-12-09 10:0

KeyboardInterrupt: ignored

# **Test**

In [ ]:
ckpt = torch.load(f"{OUTPUT}/trained_best{prefix}.pth")
# ckpt = torch.load(f"{OUTPUT}/trained_9500.pth")
model.load_state_dict(ckpt['model'])
linear = torch.nn.Identity()
batch_size = 64

In [ ]:
evaluate(model, linear, valid_loader)

In [ ]:
evaluate(model, linear, test_loader)

In [ ]:
evaluate(model, linear, train_loader)

# **Scratch**

In [ ]:
for name, param in model.named_parameters():
    if 'srate' in name:
        print(name, param.item())